In [20]:
import pandas as pd
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
import time
import os
from openai import OpenAI

# Load environment variables
load_dotenv()

True

In [13]:
token = os.getenv("RUNPOD_TOKEN")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
openai_base_url = os.getenv("RUNPOD_EMBEDIING_URL")
model_name = os.getenv("MODEL_NAME")

In [14]:
pc = Pinecone(
    api_key=pinecone_api_key)

client = OpenAI(base_url = openai_base_url, api_key=token)

In [9]:
output = client.embeddings.create(input="Hello world", model=model_name)
embedding = output.data[0].embedding
print(embedding)

[0.015215358696877956, -0.02272770367562771, 0.008572462946176529, -0.07437602430582047, 0.003935400862246752, 0.0027780423406511545, -0.03130016475915909, 0.0446622259914875, 0.04399107024073601, -0.007783094421029091, -0.02524453029036522, -0.033374641090631485, 0.014376416802406311, 0.046340107917785645, 0.00868686381727457, -0.0160466730594635, 0.007504718378186226, -0.019005851820111275, -0.11470626294612885, -0.01813640259206295, 0.1262989193201065, 0.029729057103395462, 0.025229275226593018, -0.0341678224503994, -0.04109290614724159, 0.006604762282222509, 0.010349494405090809, 0.02239212580025196, 0.004431139677762985, -0.12776325643062592, -0.016061928123235703, -0.020348157733678818, 0.04737734794616699, 0.011585026979446411, 0.06827463209629059, 0.007413197308778763, -0.018044881522655487, 0.040970880538225174, -0.010196959599852562, 0.02370392717421055, 0.010410508140921593, -0.02846301719546318, 0.008091977797448635, -0.015253492631018162, 0.03090357594192028, -0.0659561008

In [10]:
len(embedding)  # Check the length of the embedding vector

384

In [3]:
df=pd.read_json('products/products.jsonl',lines=True)
df.head(2) 

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [4]:

df['text'] =  df['name']+" : "+df['description'] + \
                " -- Ingredients: " + df['ingredients'].astype(str) + \
                " -- Price: " + df['price'].astype(str) + \
                " -- rating: " + df['rating'].astype(str)  

In [5]:
df['text'].head(2)

0    Cappuccino : A rich and creamy cappuccino made...
1    Jumbo Savory Scone : Deliciously flaky and but...
Name: text, dtype: object

In [6]:
texts = df['text'].tolist()

In [7]:
texts[0:2]  # Display the first two text entries

["Cappuccino : A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations. -- Ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 4.5 -- rating: 4.7",
 "Jumbo Savory Scone : Deliciously flaky and buttery, this jumbo savory scone is filled with herbs and cheese, creating a mouthwatering experience. Perfect for a hearty snack or a light lunch, it pairs beautifully with your favorite coffee or tea. -- Ingredients: ['Flour', 'Butter', 'Cheese', 'Herbs', 'Baking Powder', 'Salt'] -- Price: 3.25 -- rating: 4.3"]

In [8]:
with open('products/Merry\'s_way_about_us.txt') as f:
    Merry_way_about_section = f.read()
    
Merry_way_about_section = "Coffee shop Merry's Way about section: " + Merry_way_about_section
texts.append(Merry_way_about_section)

In [9]:
with open('products/menu_items_text.txt') as f:
    menue_items_text = f.read()
    
menue_items_text = "Menu Items: " + menue_items_text
texts.append(menue_items_text)

In [15]:
output = client.embeddings.create(input=texts, model=model_name)

In [16]:
embeddings = output.data

In [17]:
embeddings[0]

Embedding(embedding=[-0.014077755622565746, -0.06458218395709991, 0.0038055721670389175, 0.004829929210245609, 0.030047805979847908, -0.04565923660993576, 0.037265993654727936, 0.010659417137503624, -0.027804521843791008, -0.050939347594976425, -0.05243487283587456, -0.033664532005786896, -0.0003986789961345494, -0.043766941875219345, 0.037510160356760025, -0.017106954008340836, -0.019563883543014526, -0.080636166036129, -0.07581386715173721, -0.007958319038152695, 0.04840611666440964, -0.061224885284900665, -0.10767766088247299, -0.03699130564928055, 0.035495784133672714, -0.048467155545949936, 0.08796117454767227, -0.019823310896754265, 0.001698677777312696, -0.13075144588947296, -0.01300189457833767, 0.009842983447015285, -0.02798764780163765, -0.025835925713181496, -0.035068489611148834, 0.0009804832516238093, 0.06757322698831558, -0.07593595236539841, 0.047093719244003296, 0.0005937264650128782, -0.0025122498627752066, 0.02977311797440052, -0.06409385055303574, -0.0020410837605595

#Push the embeddings to Pinecone

In [19]:
index_name = "coffeeshop"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"))

{
    "name": "coffeeshop",
    "metric": "cosine",
    "host": "coffeeshop-73j67b0.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [21]:
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

vectors = []
for text, e in zip(texts, embeddings):
    entry_id = text.split(":")[0].strip()
    vectors.append({
        "id": entry_id,
        "values": e.embedding,
        "metadata": {'text': text}
    })
    
index.upsert(
    vectors=vectors,
    namespace="ns1"
)

/Users/yashshah/Documents/Full-Stack-AI-Chatbot-Suite--LLM--RAG--Market-Basket-Analysis/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'upserted_count': 20}

In [22]:
output = client.embeddings.create(input = ["Is Cappuccino lactose-free?"],model=model_name)
embeding = output.data[0].embedding

In [23]:
results = index.query(
    namespace="ns1",
    vector=embeding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'Cappuccino',
              'metadata': {'text': 'Cappuccino : A rich and creamy cappuccino '
                                   'made with freshly brewed espresso, steamed '
                                   'milk, and a frothy milk cap. This '
                                   'delightful drink offers a perfect balance '
                                   'of bold coffee flavor and smooth milk, '
                                   'making it an ideal companion for relaxing '
                                   'mornings or lively conversations. -- '
                                   "Ingredients: ['Espresso', 'Steamed Milk', "
                                   "'Milk Foam'] -- Price: 4.5 -- rating: 4.7"},
              'score': 0.734832764,
              'values': []},
             {'id': 'Sugar Free Vanilla syrup',
              'metadata': {'text': 'Sugar Free Vanilla syrup : Enjoy the sweet '
                                   'flavor of vanilla without the 